# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

## Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3239540 artículos de Wikipedia repartidos en 3223 archivos. Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

      ID <number>
      1234 4522 23 4 4567
      456 21 9890 123 7 0

* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
 
      palabra #ocurrencias
    
  El ID de cada palabra es el número de línea en la cual ocurre.

In [ ]:
import pandas as pd
import numpy as np
import time
import gensim
import os
from gensim.models import Word2Vec, KeyedVectors
from math import floor

from corpus import create_coarseNE_df
from corpus import spread_artID
from embeddings import fill_w2vMatrix, exponential_decay

### Cargamos los datos

In [ ]:
word_mapping = pd.read_csv('./corpus_WiNER/document.vocab', sep=' ', header=None, 
                           names=['word', 'frequency'], keep_default_na=False) 
                        # con esto evito que el parser trate como nan value a algunas palabras.

In [ ]:
print(word_mapping.shape)
word_mapping.head()

In [ ]:
len(word_mapping['word'].unique())

### Cargamos los artículos del Documento "0"

In [ ]:
doc_0 = pd.read_csv('./corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [ ]:
doc_0.shape

In [ ]:
doc_0.head()

Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [ ]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [ ]:
doc_0 = doc_0.dropna()

In [ ]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

Hacemos que cada sentencia sea una lista de palabras codificadas y casteamos a int

In [ ]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: list(map(int, x.split(' '))))

In [ ]:
doc_0.head()

Nos quedamos con un artículo

In [ ]:
article = doc_0[doc_0.art_ID == 1000]
article.head()

Reconstruimos la primera oración del artículo

In [ ]:
def sentence_decoder(sentence, word_mapping):
    dec_sentence = []
    for idx in sentence:
        mapped_w = word_mapping.loc[idx, 'word']
        dec_sentence.append(mapped_w)
    return dec_sentence 

In [ ]:
# Escribimos article.columns.get_loc('sentence') para evitar hardcodear el índice correspondiente
# a la columna 'sentence' que en este caso es 0
sentence = article.iloc[0, article.columns.get_loc('sentence')]

In [ ]:
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

### Word embeddings utilizando el modelo pre-entrenado word2vec de Google

Utilizaremos la librería Gensim https://radimrehurek.com/gensim/

Cargamos Google's pre-trained Word2Vec model.

Utilizando KeyedVectors para cargar el modelo tiene la desventaja de que no se puede seguir entrenando. Pero es más eficiente que utilizar gensim.models.Word2Vec
https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors

In [ ]:
start = time.time()
# model = KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)
# model.save('./models/word2vecGoogle.model')
w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
end = time.time()
print('demora: {}'.format(end-start))
# model = Word2Vec.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
print('Cantidad de word embeddings: {}'.format(len(w2v_model.vectors)))

In [ ]:
print('Dimensionalidad de los vectores: {}'.format(w2v_model.vector_size))

## Exploremos distintas combinaciones de vectores de palabras

### Concatenación

Este método consiste en concatenar los vectores de palabras que rodean una palabra objetivo en un vector más grande, que tiene un tamaño igual a las dimensiones agregadas de todos las proyecciones (embeddings) individuales.

- $w_{ij}$ = peso asociado con la i-ésima dimensión del vector de la j-ésima palabra en la oración. NOTA: con los vectores de palabras de una oración se forma una matriz $w^{\space D\space x\space L}$ donde $L$ es la cantidad de palabras de esa oración.
- $D$ = dimensionalidad de los word vectors originales. Por ejemplo, al usar el modelo word2vec de Google se tiene $D$ = 300.
- $W$ = tamaño de ventana que se define como el número de palabras en un solo lado.

Nos interesa representar el contexto de la I-ésima palabra de la oración. 

La i-ésima dimensión del vector de concatenación, que tiene un tamaño de $2 W D$, se calcula de la siguiente manera:

$$ e_{i} =\begin{cases} 
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space \left\lfloor{\frac{i}{D}}\right\rfloor} & \left\lfloor{\frac{i}{D}}\right\rfloor < W \\
      w_{i\space mod \space D,\space\space I \space - \space W \space + \space 1\space  +\space\left\lfloor{\frac{i}{D}}\right\rfloor} & c.c.
   \end{cases}$$
   

<br>
Al tomar una ventana simétrica, se realiza un relleno (padding) con ceros a izquierda y/o derecha según corresponda para mantener la misma dimesionalidad en cada nuevo vector generado.

### Promedio

Como su nombre indica, se calcula el centroide de los embeddings de todas las palabras circundantes. La fórmula divide cada dimensión en $2W$ ya que el número de palabras del contexto es dos veces el tamaño de la ventana:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} \frac{w_{ij}}{2W}$$

### Decaimiento fraccional

Una tercera estrategia para construir un vector de carácteristicas en base a los embeddings de palabras contextuales está inspirada en la forma en que Word2vec combina las palabras en el contexto. Aquí, se supone que la importancia de una palabra para nuestra representación es inversamente proporcional a su distancia respecto a la palabra objetivo.

Por lo tanto, las palabras contextuales se ponderan en función de su distancia de la palabra objetivo:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} *\frac{W - \lvert I-j\rvert}{W}$$


### Decaimiento exponencial

Funciona de manera similar al decaimiento fraccional, que le da más importancia al contexto cercano, pero en este caso la ponderación se realiza exponencialmente:

$$e_{i} =\sum_{\substack{j\space=\space I-W \\ j\space\neq\space I}}^{I + W} w_{ij} * (1 - \alpha)^{\lvert \space I\space-\space j\space\rvert\space-\space1}$$

donde $\alpha = 1 - 0.1^{(W-1)^{-1}}$ es el parámetro de decaimiento. Elegimos el parámetro de tal manera que las palabras inmediatas que rodean a la palabra objetivo contribuyen 10 veces más que las últimas palabras en ambos lados de la ventana.

## Exploremos CoarseNE.tar.bz2

Contiene menciones anotadas automáticamente con etiquetas de entidades nombradas (PER, LOC, ORG y MISC).

El formato es:

    ID artID
    sentIdx begin end entityType
    
donde entityType[0] = PER | entityType[1] = LOC | entityType[2] = ORG | entityType[3] = MISC

In [ ]:
coarseNE_0 = pd.read_csv('./corpus_WiNER/CoarseNE/0', sep='ID', engine='python', header=None, names=['named-entity', 'art_ID'])

In [ ]:
print(coarseNE_0.shape)
coarseNE_0.head()

Aplicamos el mismo truco que utilizamos en los documentos para propagar los art_ID según corresponda

In [ ]:
coarseNE_0 = spread_artID(coarseNE_0)
coarseNE_0 = coarseNE_0.dropna()

In [ ]:
print('coarseNE_0 contiene {} entidades.'.format(coarseNE_0.shape[0]))
print('coarseNE_0 contiene {} artículos'.format(len(coarseNE_0['art_ID'].unique())))

Creamos nuevas columnas con la info de la columna named-entity para mejor manipulación

In [ ]:
def get_tag(x):
    tags = ['PER', 'LOC', 'ORG', 'MISC']
    return tags[x]

In [ ]:
coarseNE_0['named-entity'] = coarseNE_0['named-entity'].map(lambda x: x.split('\t'))
coarseNE_0['senIdx'] = coarseNE_0['named-entity'].map(lambda x: int(x[0]))
coarseNE_0['begin'] = coarseNE_0['named-entity'].map(lambda x: int(x[1]))
coarseNE_0['end'] = coarseNE_0['named-entity'].map(lambda x: int(x[2]))
coarseNE_0['entityType'] = coarseNE_0['named-entity'].map(lambda x: get_tag(int(x[3])))
coarseNE_0 = coarseNE_0.drop(columns='named-entity')

In [ ]:
coarseNE_0.head()

No todos los artículos que ocurren en Documents/0 se encuentran en CoarseNE/0

Notar que esto sucede porque esos artículos no contienen entidades nombradas.

In [ ]:
print('Artículos que están presentes en Documents/0 pero no en CoarseNE/0: {}'
      .format(list(set(doc_0.art_ID.unique()) - set(coarseNE_0.art_ID.unique()))))

In [ ]:
article = doc_0[doc_0.art_ID == 431177]
for sentence in article.sentence.values:
    dec_sentence = sentence_decoder(sentence, word_mapping)
    print(' '.join(dec_sentence))

### Veamos como están anotadas las entidades nombradas de una oración en particular 

In [ ]:
article = doc_0[doc_0.art_ID == 1000]
sentence = article.iloc[0, article.columns.get_loc('sentence')]
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

In [ ]:
art_entities = coarseNE_0[coarseNE_0.art_ID == 1000]
entities_sen_0 = art_entities[art_entities.senIdx == 0]

In [ ]:
for idx, row in entities_sen_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

In [ ]:
doc77_df = pd.read_pickle('./corpus_WiNER/docs_df/doc_77')
article_df = doc77_df[doc77_df.art_ID == 145492]
dec_sentence = article_df.iloc[0, article_df.columns.get_loc('sentence')]

coarseNE_77 = pd.read_pickle('./corpus_WiNER/coarseNE_df/coarseNE_77')
art_entities = coarseNE_77[coarseNE_77.art_ID == 145492]
sen_entities_0 = art_entities[art_entities.senIdx == 0]

In [ ]:
def entityListFromSentence(sen_entities_df, sen_length):
    entities = []
    i = 0
    for _, row in sen_entities_df.iterrows():
        while i < row['begin']:
            entities.append('O')
            i += 1
        while i < row['end']:
            entities.append(row['entityType'])
            i += 1
    while i < sen_length:
        entities.append('O')
        i += 1
    return entities

In [ ]:
# TODO: en este metodo se podria ver de hacer el dropeo
# de algunas palabras y/o oraciones que no son/no contienen entidades

def entityListFromArticle(article_df, art_entities_df):
    art_entities_list = []
    # drop=True to avoid the old index being added as a column
    article_df = article_df.reset_index(drop=True)
    article_df['sen_length'] = article_df['sentence'].map(lambda x: len(x))
    for idx, row in article_df.iterrows():
        sen_entities_list = []
        # We take the df with the entities of each sentence
        sen_entities_df = art_entities_df[art_entities_df.senIdx == idx]
        sen_length = row['sen_length']
        # An empty dataframe means that the sentence doesn't have any entity
        if sen_entities_df.empty:
            sen_entities_list = ['O' for _ in range(sen_length)]
        else:
            sen_entities_list = entityListFromSentence(sen_entities_df, sen_length)
        art_entities_list += sen_entities_list
        
    return art_entities_list

In [ ]:
article_df.head()

In [ ]:
for idx, row in sen_entities_0.iterrows():
    print('{} : {}'.format(' '.join(dec_sentence[row['begin']:row['end']]), row['entityType']))

In [ ]:
entity_list = entityListFromArticle(article_df, art_entities)
print(' '.join(article_df['sentence'][1]))
print(entity_list[0:18])

In [ ]:
article_df = article_df.reset_index(drop=True)
article_df['sen_length'] = article_df['sentence'].map(lambda x: len(x))
article_df.sen_length.sum() == len(entity_list)

In [39]:
print('Cantidad de tokens: {}'.format(len(entity_list)))
print('Cantidad de entidades nombradas: {}'.format(len(entity_list) - entity_list.count('O')))
print('Porcentaje de entidades nombradas: {:.2f}%'.format(399/4395*100))

Cantidad de tokens: 4395
Cantidad de entidades nombradas: 399
Porcentaje de entidades nombradas: 9.08%


En general, cada artículo contiene una proporción reducida de palabras que son entidades.

Una alternativa para evitar cómputo y uso de memoria podría ser eliminar una proporción de vectores que no representan ninguna entidad (etiquetados con 'O') a la hora de construir la matriz de palabras.

In [40]:
def w2v_strategy(strategy, sentence, D, W, w2v_model):
    return {
#         'concat': concat_vectors(sentence, D, W, w2v_model), 
#         'mean': mean_vectors(sentence, D, W, w2v_model), 
#         'frac_decay': fractional_decay(sentence, D, W, w2v_model),
        'exp_decay': exponential_decay(sentence, D, W, w2v_model),
    }[strategy]

In [41]:
def wordVectorsFromArticle(article_df, strategy, D, W, w2v_model):
    count = 0
    no_vocab_list = []
    if strategy == 'concat':
        word_vectors = np.empty((0, 2*W*D))
    else:    
        word_vectors = np.empty((0, D))
    for _, row in article_df.iterrows():
        word_vec, c, l = w2v_strategy(strategy, row['sentence'], D, W, w2v_model)
        count += c
        no_vocab_list.append(l)
        word_vectors = np.append(word_vectors, word_vec, axis=0)    
    return word_vectors, count, no_vocab_list

In [42]:
w, count, no_vocab_list = wordVectorsFromArticle(article_df, strategy='exp_decay', D=300, 
                                                 W=5, w2v_model=w2v_model)
print('#palabras del artículo: {}'.format(w.shape[0]))
print('#palabras que no están en el vocabulario de w2v_model: {}'.format(count))

#palabras del artículo: 4395
#palabras que no están en el vocabulario de w2v_model: 1101


In [44]:
l = list(w)

In [ ]:
w

In [ ]:
no_vocab_list[0:10]

Muchas palabras del artículo no están en el vocabulario del modelo word2vec de Google.

En general se trata de stopwords, signos de puntuación, números, palabras mal escritas, etc.

Frente a estos casos se opta por realizar lo siguiente: si la palabra no está en el vocabulario se la transforma a un vector de 0's.

In [43]:
print(w.shape)
w_df = pd.DataFrame(w)
w_df['entityType'] = entity_list
w_e = w_df.values
print(w_e.shape)
first_component = w_e[0][0]
last_component = w_e[0][299]
entityType = w_e[0][300]
print(type(first_component), first_component)
print(type(last_component), last_component)
print(type(entityType), entityType)

(4395, 300)
(4395, 301)
<class 'float'> -0.003621502994722617
<class 'float'> 0.3130473863435913
<class 'str'> LOC


In [ ]:
#  def createWordVectors_Entity(strategy, W):
#     '''
#     Create a N x (D+1) matrix where N is the total amount of words and 
#     D is the dimensionality of each word vector. 
#     The values of the last column are the entity type (PER - LOC - ORG - MISC - O).
    
#     strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
#     W: window size
    
#     @return wordVectors_Entity matrix
#     '''
#     w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
#     D = w2v_model.vector_size
#     wordVectors_Entity = np.empty((0, D))
#     count = 0 # Used in the 'progress bar'
    
#     doc_filenames = os.listdir('./corpus_WiNER/docs_df/')
#     coarseNE_filenames = os.listdir('./corpus_WiNER/coarseNE_df/')
#     doc_filenames.sort()
#     coarseNE_filenames.sort()
    
#     for doc_f, coarseNE_f in zip(doc_filenames, coarseNE_filenames):
#         doc_df = pd.read_pickle('./corpus_WiNER/docs_df/'+ doc_f)
#         coarseNE_df = pd.read_pickle('./corpus_WiNER/coarseNE_df/'+ coarseNE_f)
#         art_IDs = coarseNE_df.art_ID.unique()
#         # We consider only the articles with at least one entity.
#         # That's why we iterate over the coarseNE's articles.
#         for art_ID in np.nditer(art_IDs):            
#             article_df = doc_df[doc_df.art_ID == art_ID]
#             m = wordVectorsFromArticle(article_df, strategy, D, W, w2v_model)
#             m = pd.DataFrame(m[0])
            
#             art_entities = coarseNE_df[coarseNE_df.art_ID == art_ID]
#             entity_list = entityListFromArticle(article_df, art_entities)            
#             m['entityType'] = entity_list
            
#             wordVectors_Entity = np.append(wordVectors_Entity, m.values)
            
#             if count % 10 == 0:
#                 print(count, end=' ')
#             count += 1
    
#     return wordVectors_Entity

In [63]:
def createWordVectors_Entity(strategy, W):
    '''  TODO: change description!
    Create a N x (D+1) matrix where N is the total amount of words and 
    D is the dimensionality of each word vector. 
    The values of the last column are the entity type (PER - LOC - ORG - MISC - O).
    
    strategy: 'concat', 'mean', 'frac_decay', 'exp_decay'.
    W: window size
    
    @return wordVectors_Entity matrix
    '''
    w2v_model = KeyedVectors.load('./models/word2vecGoogle.model')
    D = w2v_model.vector_size
    count = 0 # Used in the 'progress bar'
    
    doc_filenames = os.listdir('./corpus_WiNER/docs_df/')
    coarseNE_filenames = os.listdir('./corpus_WiNER/coarseNE_df/')
    doc_filenames.sort()
    coarseNE_filenames.sort()
    
    for doc_f, coarseNE_f in zip(doc_filenames, coarseNE_filenames):
        wordVectors = np.empty((0, D))
        entityVector = np.empty((0, 1))
        doc_df = pd.read_pickle('./corpus_WiNER/docs_df/'+ doc_f)
        coarseNE_df = pd.read_pickle('./corpus_WiNER/coarseNE_df/'+ coarseNE_f)
        art_IDs = coarseNE_df.art_ID.unique()
        # We consider only the articles with at least one entity.
        # That's why we iterate over the coarseNE's articles.
        for art_ID in np.nditer(art_IDs):            
            article_df = doc_df[doc_df.art_ID == art_ID]
            w_vectors,_,_ = wordVectorsFromArticle(article_df, strategy, D, W, w2v_model)
#             m = pd.DataFrame(m[0])
            np.savez_compressed('./corpus_WiNER/word_vectors/wv_'+doc_f, *w_vectors)
    
            art_entities = coarseNE_df[coarseNE_df.art_ID == art_ID]
            entity_list = entityListFromArticle(article_df, art_entities)            
#             m['entityType'] = entity_list
            
#             wordVectors_Entity = np.append(wordVectors_Entity, m.values)
            
            if count % 10 == 0:
                print(count, end=' ')
            count += 1

            break
        break
    print('End process')

In [64]:
start = time.time()
createWordVectors_Entity(strategy='exp_decay', W=5)
end = time.time()
print('Demora: {}'.format(end - start))

0 End process
Demora: 13.359838962554932


In [65]:
word_vectors = np.load('./corpus_WiNER/word_vectors/wv_doc_0.npz')

In [61]:
items2 = word_vectors.items()

In [66]:
for a_name, arr in word_vectors.iteritems():
    print(a_name, len(arr))

arr_0 300
arr_1 300
arr_2 300
arr_3 300
arr_4 300
arr_5 300
arr_6 300
arr_7 300
arr_8 300
arr_9 300
arr_10 300
arr_11 300
arr_12 300
arr_13 300
arr_14 300
arr_15 300
arr_16 300
arr_17 300
arr_18 300
arr_19 300
arr_20 300
arr_21 300
arr_22 300
arr_23 300
arr_24 300
arr_25 300
arr_26 300
arr_27 300
arr_28 300
arr_29 300
arr_30 300
arr_31 300
arr_32 300
arr_33 300
arr_34 300
arr_35 300
arr_36 300
arr_37 300
arr_38 300
arr_39 300
arr_40 300
arr_41 300
arr_42 300
arr_43 300
arr_44 300
arr_45 300
arr_46 300
arr_47 300
arr_48 300
arr_49 300
arr_50 300
arr_51 300
arr_52 300
arr_53 300
arr_54 300
arr_55 300
arr_56 300
arr_57 300
arr_58 300
arr_59 300
arr_60 300
arr_61 300
arr_62 300
arr_63 300
arr_64 300
arr_65 300
arr_66 300
arr_67 300
arr_68 300
arr_69 300
arr_70 300
arr_71 300
arr_72 300
arr_73 300
arr_74 300
arr_75 300
arr_76 300
arr_77 300
arr_78 300
arr_79 300
arr_80 300
arr_81 300
arr_82 300
arr_83 300
arr_84 300
arr_85 300
arr_86 300
arr_87 300
arr_88 300
arr_89 300
arr_90 300
arr_91 30

arr_868 300
arr_869 300
arr_870 300
arr_871 300
arr_872 300
arr_873 300
arr_874 300
arr_875 300
arr_876 300
arr_877 300
arr_878 300
arr_879 300
arr_880 300
arr_881 300
arr_882 300
arr_883 300
arr_884 300
arr_885 300
arr_886 300
arr_887 300
arr_888 300
arr_889 300
arr_890 300
arr_891 300
arr_892 300
arr_893 300
arr_894 300
arr_895 300
arr_896 300
arr_897 300
arr_898 300
arr_899 300
arr_900 300
arr_901 300
arr_902 300
arr_903 300
arr_904 300
arr_905 300
arr_906 300
arr_907 300
arr_908 300
arr_909 300
arr_910 300
arr_911 300
arr_912 300
arr_913 300
arr_914 300
arr_915 300
arr_916 300
arr_917 300
arr_918 300
arr_919 300
arr_920 300
arr_921 300
arr_922 300
arr_923 300
arr_924 300
arr_925 300
arr_926 300
arr_927 300
arr_928 300
arr_929 300
arr_930 300
arr_931 300
arr_932 300
arr_933 300
arr_934 300
arr_935 300
arr_936 300
arr_937 300
arr_938 300
arr_939 300
arr_940 300
arr_941 300
arr_942 300
arr_943 300
arr_944 300
arr_945 300
arr_946 300
arr_947 300
arr_948 300
arr_949 300
arr_950 300
arr_

arr_1685 300
arr_1686 300
arr_1687 300
arr_1688 300
arr_1689 300
arr_1690 300
arr_1691 300
arr_1692 300
arr_1693 300
arr_1694 300
arr_1695 300
arr_1696 300
arr_1697 300
arr_1698 300
arr_1699 300
arr_1700 300
arr_1701 300
arr_1702 300
arr_1703 300
arr_1704 300
arr_1705 300
arr_1706 300
arr_1707 300
arr_1708 300
arr_1709 300
arr_1710 300
arr_1711 300
arr_1712 300
arr_1713 300
arr_1714 300
arr_1715 300
arr_1716 300
arr_1717 300
arr_1718 300
arr_1719 300
arr_1720 300
arr_1721 300
arr_1722 300
arr_1723 300
arr_1724 300
arr_1725 300
arr_1726 300
arr_1727 300
arr_1728 300
arr_1729 300
arr_1730 300
arr_1731 300
arr_1732 300
arr_1733 300
arr_1734 300
arr_1735 300
arr_1736 300
arr_1737 300
arr_1738 300
arr_1739 300
arr_1740 300
arr_1741 300
arr_1742 300
arr_1743 300
arr_1744 300
arr_1745 300
arr_1746 300
arr_1747 300
arr_1748 300
arr_1749 300
arr_1750 300
arr_1751 300
arr_1752 300
arr_1753 300
arr_1754 300
arr_1755 300
arr_1756 300
arr_1757 300
arr_1758 300
arr_1759 300
arr_1760 300
arr_1761 300

arr_2420 300
arr_2421 300
arr_2422 300
arr_2423 300
arr_2424 300
arr_2425 300
arr_2426 300
arr_2427 300
arr_2428 300
arr_2429 300
arr_2430 300
arr_2431 300
arr_2432 300
arr_2433 300
arr_2434 300
arr_2435 300
arr_2436 300
arr_2437 300
arr_2438 300
arr_2439 300
arr_2440 300
arr_2441 300
arr_2442 300
arr_2443 300
arr_2444 300
arr_2445 300
arr_2446 300
arr_2447 300
arr_2448 300
arr_2449 300
arr_2450 300
arr_2451 300
arr_2452 300
arr_2453 300
arr_2454 300
arr_2455 300
arr_2456 300
arr_2457 300
arr_2458 300
arr_2459 300
arr_2460 300
arr_2461 300
arr_2462 300
arr_2463 300
arr_2464 300
arr_2465 300
arr_2466 300
arr_2467 300
arr_2468 300
arr_2469 300
arr_2470 300
arr_2471 300
arr_2472 300
arr_2473 300
arr_2474 300
arr_2475 300
arr_2476 300
arr_2477 300
arr_2478 300
arr_2479 300
arr_2480 300
arr_2481 300
arr_2482 300
arr_2483 300
arr_2484 300
arr_2485 300
arr_2486 300
arr_2487 300
arr_2488 300
arr_2489 300
arr_2490 300
arr_2491 300
arr_2492 300
arr_2493 300
arr_2494 300
arr_2495 300
arr_2496 300

arr_3268 300
arr_3269 300
arr_3270 300
arr_3271 300
arr_3272 300
arr_3273 300
arr_3274 300
arr_3275 300
arr_3276 300
arr_3277 300
arr_3278 300
arr_3279 300
arr_3280 300
arr_3281 300
arr_3282 300
arr_3283 300
arr_3284 300
arr_3285 300
arr_3286 300
arr_3287 300
arr_3288 300
arr_3289 300
arr_3290 300
arr_3291 300
arr_3292 300
arr_3293 300
arr_3294 300
arr_3295 300
arr_3296 300
arr_3297 300
arr_3298 300
arr_3299 300
arr_3300 300
arr_3301 300
arr_3302 300
arr_3303 300
arr_3304 300
arr_3305 300
arr_3306 300
arr_3307 300
arr_3308 300
arr_3309 300
arr_3310 300
arr_3311 300
arr_3312 300
arr_3313 300
arr_3314 300
arr_3315 300
arr_3316 300
arr_3317 300
arr_3318 300
arr_3319 300
arr_3320 300
arr_3321 300
arr_3322 300
arr_3323 300
arr_3324 300
arr_3325 300
arr_3326 300
arr_3327 300
arr_3328 300
arr_3329 300
arr_3330 300
arr_3331 300
arr_3332 300
arr_3333 300
arr_3334 300
arr_3335 300
arr_3336 300
arr_3337 300
arr_3338 300
arr_3339 300
arr_3340 300
arr_3341 300
arr_3342 300
arr_3343 300
arr_3344 300

arr_4060 300
arr_4061 300
arr_4062 300
arr_4063 300
arr_4064 300
arr_4065 300
arr_4066 300
arr_4067 300
arr_4068 300
arr_4069 300
arr_4070 300
arr_4071 300
arr_4072 300
arr_4073 300
arr_4074 300
arr_4075 300
arr_4076 300
arr_4077 300
arr_4078 300
arr_4079 300
arr_4080 300
arr_4081 300
arr_4082 300
arr_4083 300
arr_4084 300
arr_4085 300
arr_4086 300
arr_4087 300
arr_4088 300
arr_4089 300
arr_4090 300
arr_4091 300
arr_4092 300
arr_4093 300
arr_4094 300
arr_4095 300
arr_4096 300
arr_4097 300
arr_4098 300
arr_4099 300
arr_4100 300
arr_4101 300
arr_4102 300
arr_4103 300
arr_4104 300
arr_4105 300
arr_4106 300
arr_4107 300
arr_4108 300
arr_4109 300
arr_4110 300
arr_4111 300
arr_4112 300
arr_4113 300
arr_4114 300
arr_4115 300
arr_4116 300
arr_4117 300
arr_4118 300
arr_4119 300
arr_4120 300
arr_4121 300
arr_4122 300
arr_4123 300
arr_4124 300
arr_4125 300
arr_4126 300
arr_4127 300
arr_4128 300
arr_4129 300
arr_4130 300
arr_4131 300
arr_4132 300
arr_4133 300
arr_4134 300
arr_4135 300
arr_4136 300

arr_4867 300
arr_4868 300
arr_4869 300
arr_4870 300
arr_4871 300
arr_4872 300
arr_4873 300
arr_4874 300
arr_4875 300
arr_4876 300
arr_4877 300
arr_4878 300
arr_4879 300
arr_4880 300
arr_4881 300
arr_4882 300
arr_4883 300
arr_4884 300
arr_4885 300
arr_4886 300
arr_4887 300
arr_4888 300
arr_4889 300
arr_4890 300
arr_4891 300
arr_4892 300
arr_4893 300
arr_4894 300
arr_4895 300
arr_4896 300
arr_4897 300
arr_4898 300
arr_4899 300
arr_4900 300
arr_4901 300
arr_4902 300
arr_4903 300
arr_4904 300
arr_4905 300
arr_4906 300
arr_4907 300
arr_4908 300
arr_4909 300
arr_4910 300
arr_4911 300
arr_4912 300
arr_4913 300
arr_4914 300
arr_4915 300
arr_4916 300
arr_4917 300
arr_4918 300
arr_4919 300
arr_4920 300
arr_4921 300
arr_4922 300
arr_4923 300
arr_4924 300
arr_4925 300
arr_4926 300
arr_4927 300
arr_4928 300
arr_4929 300
arr_4930 300
arr_4931 300
arr_4932 300
arr_4933 300
arr_4934 300
arr_4935 300
arr_4936 300
arr_4937 300
arr_4938 300
arr_4939 300
arr_4940 300
arr_4941 300
arr_4942 300
arr_4943 300

arr_5766 300
arr_5767 300
arr_5768 300
arr_5769 300
arr_5770 300
arr_5771 300
arr_5772 300
arr_5773 300
arr_5774 300
arr_5775 300
arr_5776 300
arr_5777 300
arr_5778 300
arr_5779 300
arr_5780 300
arr_5781 300
arr_5782 300
arr_5783 300
arr_5784 300
arr_5785 300
arr_5786 300
arr_5787 300
arr_5788 300
arr_5789 300
arr_5790 300
arr_5791 300
arr_5792 300
arr_5793 300
arr_5794 300
arr_5795 300
arr_5796 300
arr_5797 300
arr_5798 300
arr_5799 300
arr_5800 300
arr_5801 300
arr_5802 300
arr_5803 300
arr_5804 300
arr_5805 300
arr_5806 300
arr_5807 300
arr_5808 300
arr_5809 300
arr_5810 300
arr_5811 300
arr_5812 300
arr_5813 300
arr_5814 300
arr_5815 300
arr_5816 300
arr_5817 300
arr_5818 300
arr_5819 300
arr_5820 300
arr_5821 300
arr_5822 300
arr_5823 300
arr_5824 300
arr_5825 300
arr_5826 300
arr_5827 300
arr_5828 300
arr_5829 300
arr_5830 300
arr_5831 300
arr_5832 300
arr_5833 300
arr_5834 300
arr_5835 300
arr_5836 300
arr_5837 300
arr_5838 300
arr_5839 300
arr_5840 300
arr_5841 300
arr_5842 300

arr_6536 300
arr_6537 300
arr_6538 300
arr_6539 300
arr_6540 300
arr_6541 300
arr_6542 300
arr_6543 300
arr_6544 300
arr_6545 300
arr_6546 300
arr_6547 300
arr_6548 300
arr_6549 300
arr_6550 300
arr_6551 300
arr_6552 300
arr_6553 300
arr_6554 300
arr_6555 300
arr_6556 300
arr_6557 300
arr_6558 300
arr_6559 300
arr_6560 300
arr_6561 300
arr_6562 300
arr_6563 300
arr_6564 300
arr_6565 300
arr_6566 300
arr_6567 300
arr_6568 300
arr_6569 300
arr_6570 300
arr_6571 300
arr_6572 300
arr_6573 300
arr_6574 300
arr_6575 300
arr_6576 300
arr_6577 300
arr_6578 300
arr_6579 300
arr_6580 300
arr_6581 300
arr_6582 300
arr_6583 300
arr_6584 300
arr_6585 300
arr_6586 300
arr_6587 300
arr_6588 300
arr_6589 300
arr_6590 300
arr_6591 300
arr_6592 300
arr_6593 300
arr_6594 300
arr_6595 300
arr_6596 300
arr_6597 300
arr_6598 300
arr_6599 300
arr_6600 300
arr_6601 300
arr_6602 300
arr_6603 300
arr_6604 300
arr_6605 300
arr_6606 300
arr_6607 300
arr_6608 300
arr_6609 300
arr_6610 300
arr_6611 300
arr_6612 300

arr_7256 300
arr_7257 300
arr_7258 300
arr_7259 300
arr_7260 300
arr_7261 300
arr_7262 300
arr_7263 300
arr_7264 300
arr_7265 300
arr_7266 300
arr_7267 300
arr_7268 300
arr_7269 300
arr_7270 300
arr_7271 300
arr_7272 300
arr_7273 300
arr_7274 300
arr_7275 300
arr_7276 300
arr_7277 300
arr_7278 300
arr_7279 300
arr_7280 300
arr_7281 300
arr_7282 300
arr_7283 300
arr_7284 300
arr_7285 300
arr_7286 300
arr_7287 300
arr_7288 300
arr_7289 300
arr_7290 300
arr_7291 300
arr_7292 300
arr_7293 300
arr_7294 300
arr_7295 300
arr_7296 300
arr_7297 300
arr_7298 300
arr_7299 300
arr_7300 300
arr_7301 300
arr_7302 300
arr_7303 300
arr_7304 300
arr_7305 300
arr_7306 300
arr_7307 300
arr_7308 300
arr_7309 300
arr_7310 300
arr_7311 300
arr_7312 300
arr_7313 300
arr_7314 300
arr_7315 300
arr_7316 300
arr_7317 300
arr_7318 300
arr_7319 300
arr_7320 300
arr_7321 300
arr_7322 300
arr_7323 300
arr_7324 300
arr_7325 300
arr_7326 300
arr_7327 300
arr_7328 300
arr_7329 300
arr_7330 300
arr_7331 300
arr_7332 300

In [ ]:
items

In [ ]:
len(df.loc[0,1])

Cantidad de oraciones: 54607542
Cantidad de artículos: 3239540